In [ ]:
from pathlib import Path

examples_folder = Path('~/Projects/crosscompute-examples').expanduser()
automation_folder = configuration_folder = examples_folder / 'reports/randomize-histograms'
configuration_path = configuration_folder / 'automate.yml'
configuration_path

In [ ]:
from crosscompute.routines.configuration import *
from crosscompute.routines.validation import *

configuration = load_configuration(configuration_path)
raw_batch_definitions = configuration.get('batches', [])
raw_batch_definitions

In [ ]:
raw_batch_definition = BatchDefinition(raw_batch_definitions[2])
raw_batch_definition

In [ ]:
reference_folder = raw_batch_definition['reference']['folder']
reference_folder

In [ ]:
(configuration.folder / reference_folder).exists()
# TODO: Check that reference folder exists

In [ ]:
variable_definitions = configuration.get_variable_definitions('input')
variable_definitions

In [ ]:
data_by_id = {}
variable_definition = variable_definitions[0]
variable_definition

In [ ]:
variable_definition.__dict__

In [ ]:
join(configuration.folder, reference_folder)

In [ ]:
variable_definition.path

In [ ]:
from crosscompute.routines.variable import *
load_variable_data(
    configuration.folder / reference_folder / 'input' / variable_definition.path,
    variable_definition.id)

In [ ]:
def get_data_by_id_from_folder(folder, variable_definitions):
    data_by_id = {}
    for variable_definition in variable_definitions:
        variable_id = variable_definition.id
        variable_path = variable_definition.path
        variable_data = load_variable_data(folder / variable_path, variable_id)
        data_by_id[variable_id] = variable_data
    return data_by_id

variable_definitions = configuration.get_variable_definitions('input')
reference_data_by_id = get_data_by_id_from_folder(
    configuration.folder / reference_folder / 'input',
    variable_definitions)
reference_data_by_id

In [ ]:
raw_batch_definition

In [ ]:
from crosscompute.routines.validation import *

In [ ]:
batch_configuration_path = raw_batch_definition['configuration']['path']
batch_configuration_path

In [ ]:
import csv

def yield_data_by_id_from_csv(path, variable_definitions):
    try:
        with path.open('rt') as f:
            csv_reader = csv.reader(f)
            keys = [_.strip() for _ in next(csv_reader)]
            for values in csv_reader:
                data_by_id = {k: {'value': v} for k, v in zip(keys, values)}
                data_by_id = parse_data_by_id(data_by_id, variable_definitions)
                if data_by_id.get('#') == '#':
                    continue
                yield data_by_id
    except OSError:
        raise CrossComputeConfigurationError(f'{path} path not found')

In [ ]:
list(yield_data_by_id_from_csv(automation_folder / batch_configuration_path, variable_definitions))

In [ ]:
configuration_data_by_id = next(yield_data_by_id_from_csv(automation_folder / batch_configuration_path, variable_definitions))
configuration_data_by_id

In [ ]:
reference_data_by_id.update(configuration_data_by_id)
reference_data_by_id

In [ ]:
data_by_id = reference_data_by_id | configuration_data_by_id
data_by_id

In [ ]:
raw_batch_folder = raw_batch_definition.folder
raw_batch_name = raw_batch_definition.name
raw_batch_slug = raw_batch_definition.slug
raw_batch_definition.__dict__

In [ ]:
data_by_id

In [ ]:
str(raw_batch_folder)

In [ ]:
batch_folder = format_text(str(raw_batch_folder), data_by_id)
batch_folder

In [ ]:
batch_name = format_text(raw_batch_name, data_by_id)
batch_name

In [ ]:
batch_slug = format_text(raw_batch_slug, data_by_id)
batch_slug

In [ ]:
def get_batch_definitions_from_path(
        batch_configuration_path, raw_batch_definition, variable_definitions):
    if 'folder' in raw_batch_definition.reference:
        pass
    return []

In [ ]:
def get_batch_definitions(
        raw_batch_definition, automation_folder, variable_definitions):
    batch_definitions = []
    raw_batch_definition = BatchDefinition(raw_batch_definition)

    if 'folder' in raw_batch_definition.reference:
        reference_folder = raw_batch_definition.reference['folder']
        reference_data_by_id = get_data_by_id_from_folder(
            automation_folder / reference_folder / 'input',
            variable_definitions)
    else:
        reference_data_by_id = {}

    if 'path' in raw_batch_definition.configuration:
        batch_configuration_path = raw_batch_definition.configuration['path']
        for configuration_data_by_id in yield_data_by_id_from_csv(
                automation_folder / batch_configuration_path, variable_definitions):
            batch_definitions.append(BatchDefinition(
                raw_batch_definition,
                data_by_id=reference_data_by_id | configuration_data_by_id))
    else:
        batch_definitions.append(BatchDefinition(
            raw_batch_definition,
            data_by_id=reference_data_by_id))
    return batch_definitions

get_batch_definitions(
    raw_batch_definition, automation_folder, variable_definitions)

In [ ]:
# If a configuration exists, load data_by_id from it
# If folder exists, load anything missing from it
# If reference exists, load remaining
# Check for missing variables
# Save extra variables in variables.dictionary

In [ ]:
# 1. reference folder
# 3. configuration path